In [14]:
#Dependencies
from sklearn.preprocessing import RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np
from scipy.io import arff

In the EDA, it is seen that the amount of Cement, Superplaticizer and Age affected the strength of concrete most strongly.\
Ratios of these materials to water are possible avenues for feature engineering.

In [10]:
arff_file = arff.loadarff('/workspaces/2025SE-Mike.N-HSC-AT2/data/cement.arff')
X_full = pd.DataFrame(arff_file[0])

In [11]:
X_full.describe()

,cement,blast_furnace_slag,fly_ash,water,superplasticizer,coarse_aggregate,fine_aggregate,age,strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.165631,73.895485,54.187136,181.566359,6.203112,972.918592,773.578883,45.662136,35.817836
std,104.507142,86.279104,63.996469,21.355567,5.973492,77.753818,80.175427,63.169912,16.705679
min,102.000000,0.000000,0.000000,121.750000,0.000000,801.000000,594.000000,1.000000,2.331808
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.707115
50%,272.900000,22.000000,0.000000,185.000000,6.350000,968.000000,779.510000,28.000000,34.442774
75%,350.000000,142.950000,118.270000,192.000000,10.160000,1029.400000,824.000000,56.000000,46.136287
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.599225


In [12]:
X_full['cement_to_water_ratio'] = X_full['cement'] / X_full['water']
X_full['superplasticizer_to_water_ratio'] = X_full['superplasticizer'] / X_full['water']
X_full['fine_aggregate_to_water_ratio'] = X_full['fine_aggregate'] / X_full['water']
X_full['cement_to_fly_ash_ratio'] = X_full['cement'] / X_full['fly_ash']
X_full['superplasticizer_to_fly_ash_ratio'] = X_full['superplasticizer'] / X_full['fly_ash']
X_full['age_squared'] = X_full['age'] ** 2

These pairs of features have high correlation, shown in the EDA step.

In [13]:
X_full.describe()

/home/codespace/.local/lib/python3.12/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/home/codespace/.python/current/lib/python3.12/site-packages/numpy/lib/function_base.py:4655: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
/home/codespace/.local/lib/python3.12/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/home/codespace/.python/current/lib/python3.12/site-packages/numpy/lib/function_base.py:4655: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


,cement,blast_furnace_slag,fly_ash,water,superplasticizer,coarse_aggregate,fine_aggregate,age,strength,cement_to_water_ratio,superplasticizer_to_water_ratio,fine_aggregate_to_water_ratio,cement_to_fly_ash_ratio,superplasticizer_to_fly_ash_ratio,age_squared
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,657.000000,1030.000000
mean,281.165631,73.895485,54.187136,181.566359,6.203112,972.918592,773.578883,45.662136,35.817836,1.578275,0.037402,4.344763,inf,inf,6071.594175
std,104.507142,86.279104,63.996469,21.355567,5.973492,77.753818,80.175427,63.169912,16.705679,0.648105,0.039131,0.824908,NaN,NaN,20195.954706
min,102.000000,0.000000,0.000000,121.750000,0.000000,801.000000,594.000000,1.000000,2.331808,0.531250,0.000000,2.605263,0.699482,0.000000,1.000000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.707115,1.069502,0.000000,3.888350,2.125391,0.061166,49.000000
50%,272.900000,22.000000,0.000000,185.000000,6.350000,968.000000,779.510000,28.000000,34.442774,1.480718,0.034663,4.299479,NaN,0.091275,784.000000
75%,350.000000,142.950000,118.270000,192.000000,10.160000,1029.400000,824.000000,56.000000,46.136287,1.875000,0.059592,4.791273,NaN,NaN,3136.000000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.599225,3.746827,0.233672,7.840442,inf,inf,133225.000000


0s causing some of the ratios to be inf or -inf. We impute these values with 0s.

In [15]:
X_full.replace([np.inf, -np.inf], 0, inplace=True)

In [16]:
X_full.describe()

,cement,blast_furnace_slag,fly_ash,water,superplasticizer,coarse_aggregate,fine_aggregate,age,strength,cement_to_water_ratio,superplasticizer_to_water_ratio,fine_aggregate_to_water_ratio,cement_to_fly_ash_ratio,superplasticizer_to_fly_ash_ratio,age_squared
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,657.000000,1030.000000
mean,281.165631,73.895485,54.187136,181.566359,6.203112,972.918592,773.578883,45.662136,35.817836,1.578275,0.037402,4.344763,1.074774,0.059260,6071.594175
std,104.507142,86.279104,63.996469,21.355567,5.973492,77.753818,80.175427,63.169912,16.705679,0.648105,0.039131,0.824908,1.665946,0.062538,20195.954706
min,102.000000,0.000000,0.000000,121.750000,0.000000,801.000000,594.000000,1.000000,2.331808,0.531250,0.000000,2.605263,0.000000,0.000000,1.000000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.707115,1.069502,0.000000,3.888350,0.000000,0.000000,49.000000
50%,272.900000,22.000000,0.000000,185.000000,6.350000,968.000000,779.510000,28.000000,34.442774,1.480718,0.034663,4.299479,0.000000,0.057791,784.000000
75%,350.000000,142.950000,118.270000,192.000000,10.160000,1029.400000,824.000000,56.000000,46.136287,1.875000,0.059592,4.791273,1.905525,0.083333,3136.000000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.599225,3.746827,0.233672,7.840442,11.332379,0.457482,133225.000000


In [18]:
# Export the data
X_full.to_csv('/workspaces/2025SE-Mike.N-HSC-AT2/data/concrete_engineered.csv', index=False)